# Annual GDP growth — EDA and Starter Notebook
This notebook loads the dataset `dataset/real-gdp-growth.csv` and provides an end-to-end starting point: installation, basic EDA, cleaning, visualizations, quick ML pipeline, and a project roadmap.

## 1) Instalar dependencias

Ejecutar en terminal (recomendado usar el venv del workspace):

```bash
python -m pip install -r ../requirements.txt
# o instalar paquetes individualmente
python -m pip install pandas numpy matplotlib seaborn plotly altair scikit-learn statsmodels jupyterlab streamlit
# Opcional: context7-mcp para integración con MCP
python -m pip install context7-mcp
```


## 2) Configurar entorno

- Selecciona el kernel de Python del entorno virtual 'datadatadata/.venv' en VSCode.
- Si no tienes un venv, crea uno:

```bash
python -m venv .venv
.\.venv\Scripts\activate
python -m pip install -r requirements.txt
```

- En VSCode: `Python: Select Interpreter` → Escoge el venv.


In [ ]:
## 3) Leer y mostrar dataset/readme.md

from pathlib import Path
from IPython.display import Markdown

readme_path = Path('../dataset/readme.md')
print('README path exists:', readme_path.exists())
# Use utf-8 encoding to avoid decode errors
try:
    content = readme_path.read_text(encoding='utf-8')
    Markdown(content)
except Exception as e:
    print(f'Error reading readme: {e}')

In [ ]:
## 4) Descubrir archivos del dataset (paths, formatos)

from pathlib import Path
for p in sorted(Path('../dataset').glob('*')):
    print(p.name)


In [ ]:
## 5) Cargar datos en pandas (CSV/JSON/Parquet)

import pandas as pd

df = pd.read_csv('../dataset/real-gdp-growth.csv')
print('Shape:', df.shape)
df.head()


In [ ]:
## 6) Inspección preliminar y estadística básica

# Renombrar algunas columnas para trabajar más fácilmente
COL_MAP = {
    'Gross domestic product, constant prices - Percent change - Observations': 'gdp_obs',
    'Gross domestic product, constant prices - Percent change - Forecasts': 'gdp_forecast'
}

df = df.rename(columns=COL_MAP)

print(df.info())
print('\nResumen numérico:')
print(df.describe())

# Valores únicos y cardinalidades
print('\nNúmero de entidades:', df['Entity'].nunique())
print('Años:', df['Year'].min(), '-', df['Year'].max())


In [ ]:
## 7) Limpieza y transformación

# Convertir Year a entero si es necesario
df['Year'] = df['Year'].astype(int)
# Convertir columnas numéricas
for c in ['gdp_obs','gdp_forecast']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Mostrar nulos por columna
print(df[['Entity','Year','gdp_obs','gdp_forecast']].isna().sum())

# Estrategias simples: rellenar forecast con forward-fill por entidad
df = df.sort_values(['Entity','Year'])

df['gdp_forecast'] = df.groupby('Entity')['gdp_forecast'].apply(lambda x: x.ffill())

# Opcional: crear columna con último valor disponible



In [ ]:
## 8) Análisis exploratorio visual (gráficos estáticos)

import matplotlib.pyplot as plt
import seaborn as sns

# Distribución global de gdp_obs
plt.figure(figsize=(8,4))
sns.histplot(df['gdp_obs'].dropna(), bins=60)
plt.title('Distribución de cambios porcentuales (gdp_obs)')
plt.xlabel('Percent change')
plt.show()

# Series de tiempo para algunos países
countries = ['United States','China','India','Germany','Japan']
s = df[df['Entity'].isin(countries)]
plt.figure(figsize=(10,5))
for c in countries:
    sub = s[s['Entity']==c]
    plt.plot(sub['Year'], sub['gdp_obs'], label=c)
plt.legend()
plt.title('GDP growth — seleccionados')
plt.ylabel('% change')
plt.show()


In [ ]:
## 9) Visualizaciones interactivas (Plotly)

import plotly.express as px

# Line plot interactivo para US/China/India
sel = df[df['Entity'].isin(countries)]
fig = px.line(sel, x='Year', y='gdp_obs', color='Entity', title='GDP growth interactive')
fig.update_traces(mode='lines+markers')
fig.show()

# Top countries in latest year
year = df['Year'].max()
latest = df[df['Year']==year].dropna(subset=['gdp_obs']).sort_values('gdp_obs', ascending=False).head(20)
fig2 = px.bar(latest, x='gdp_obs', y='Entity', orientation='h', title=f'Top GDP growth ({year})')
fig2.show()


In [ ]:
## 10) Ingeniería de características y preprocesamiento

from sklearn.preprocessing import StandardScaler

# Por ejemplo: crear media móvil 3 años para suavizar la serie

df['gdp_obs_rolling3'] = df.groupby('Entity')['gdp_obs'].rolling(3, min_periods=1).mean().reset_index(level=0, drop=True)

# Agregar identificadores numéricos para entidades
entity_encoder = {e: i for i, e in enumerate(df['Entity'].unique())}
df['entity_id'] = df['Entity'].map(entity_encoder)

scaler = StandardScaler()
df['gdp_obs_scaled'] = scaler.fit_transform(df[['gdp_obs_rolling3']].fillna(0))

print('Feature examples added: gdp_obs_rolling3, entity_id, gdp_obs_scaled')


In [ ]:
## 11) Pipeline rápido de ML y evaluación (ejemplo clustering)

from sklearn.cluster import KMeans

# Cluster across entities by mean growth in the latest 10 years
group = df[df['Year']>= (df['Year'].max() - 9)].groupby('Entity')['gdp_obs'].mean().dropna().reset_index()
X = group[['gdp_obs']]
km = KMeans(n_clusters=3, random_state=42).fit(X)
group['cluster'] = km.labels_

print(group.sort_values('gdp_obs', ascending=False).head(10))

# Merge cluster back to main df
cluster_map = dict(zip(group['Entity'], group['cluster']))
df['cluster'] = df['Entity'].map(cluster_map)



## 12) Uso opcional de context7 mcp (instalación y anotaciones)

# Ejemplo de instalación: `pip install context7-mcp`
# Configura variables de entorno con tu API key si vas a usar servicios MCP.

# Código de ejemplo (placeholder):
# from context7_mcp import Client
# client = Client(api_key=os.environ.get('CONTEXT7_API_KEY'))
# client.create_annotation(...)

print('Context7 MCP: opcional — configura API keys en variables de entorno si lo usas')

In [ ]:
## 13) Exportar resultados y reproducibilidad

# Guardar un dataset procesado
processed_path = '../dataset/real-gdp-growth-processed.parquet'
df.to_parquet(processed_path, index=False)
print('Procesado guardado en', processed_path)

# Guardar el modelo de clustering (simple ejemplo)
import joblib
joblib.dump(km, '../models/kmeans_gdp.joblib')
print('KMeans guardado en models/kmeans_gdp.joblib')

# Convertir notebook a HTML/Markdown para documentación: `jupyter nbconvert --to html notebooks/gdp_eda.ipynb`


## 14) Roadmap del proyecto

1. Definir objetivo del proyecto (p.ej., "Analizar drivers de crecimiento y clusters por región").
2. EDA profunda: estacionalidad, outliers, series por país, agregaciones.
3. Limpieza avanzada: imputación basada en fuentes, fusiones con otros datasets (población, empleo, inversión).
4. Ingeniería de características: lag features, variables macroeconómicas, indicadores compuestos.
5. Modelado: clustering, forecast (ARIMA/ETS/statsmodels/LightGBM), análisis de sensibilidad.
6. Visualización y producto: dashboard con filtrado por país/region y comparaciones históricas.
7. Documentation + tests: reproducir análisis, empaquetar pipeline en scripts y CI.
8. Deployment: desplegar app Streamlit o API para el dashboard y servir los datasets procesados.

Tiempos estimados por sprint: 1-2 semanas por fase, total 8-12 semanas para un MVP.
